In [1]:
import pandas as pd
import numpy as np

In [2]:
from IPython.core.display import display, HTML # Расширить рабочее поле ноутбука на весь экран
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
from IPython.core.interactiveshell import InteractiveShell # Включить возможность вывести несколько output-ов в одной ячейке
InteractiveShell.ast_node_interactivity = "all"

Довольно часто возникает необходимость объединить данные из различных источников. Это можно сделать с помощью обычной конкатенации наборов данных, или через соединение и слияние в стиле баз данных. Объекты Series и DataFrame созданы в расчете на подолбные операции, и бибилиотека Pandas содержит функции и методы для быстрого и удобного выполнения таких манипуляций

https://pandas.pydata.org/pandas-docs/version/0.23.4/merging.html

Для удобства опишем функцию, создающую объект DataFrame определенной фромы. Эта функция пригодится нам в дальнейшем

In [5]:
def make_df(cols, inds):
    """Быстро создаем объект DataFrame"""
    
    data = {
        col: [str(col) + str(ind) for ind in inds] for col in cols
    }
    
    return pd.DataFrame(data, inds)

make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


# Конкатинация - pd.concat
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html - ссылка на документацию

Функцию pd.concat можно использовать для простой конкатенации объектов Series или DataFrame аналогично тому, как функцию np.concatenatie() можно применять для простой конкатенации массивов:

In [6]:
series1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
series2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])

pd.concat([series1, series2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

Она также подходить для конкатенации объектов более высокой размерности, таких как DataFrame:

In [16]:
df1 = make_df('AB', [1,2])
df2 = make_df('AB', [3,4])

df1
df2
pd.concat([df1, df2])

,A,B
1,A1,B1
2,A2,B2


,A,B
3,A3,B3
4,A4,B4


,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


По умолчанию конкатенация происходит построчно, то есть axis=0. Однако, при необходимости, мы можем изменить ось: 

In [19]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])

df3
df4
pd.concat([df3, df4], 
          axis='columns' # аналогично axis=1
)

,A,B
0,A0,B0
1,A1,B1


,C,D
0,C0,D0
1,C1,D1


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1


## Дублирование индексов

Конкатенация библиотеки Pandas **сохраняет индексы**, даже если в результате некоторые индексы будут дублироваться. Рассмотрим следующий пример:

In [22]:
x = make_df('AB', [0,1])
y = make_df('AB', [2, 3])

y.index = x.index # дублируем индексы

x
y
pd.concat([x, y])

,A,B
0,A0,B0
1,A1,B1


,A,B
0,A2,B2
1,A3,B3


,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Обратите внимание на повторяющиеся индексы. Хотя в объектах DataFrame это допустимо, подобный резуьлтат часто может быть нежелателен. Функция pd.concat() предоста 

### Перехват повторов как ошибки

Если вам нужно просто гарантировать, что индексы в возвращаемом функцией pd.concat() результате не перекрываются, можно задать флаг verify_integrity=True. Тогда конкатинация приведет к генрации ошибки при наличии дублирующихся индексов. 

In [25]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


### Игнорирование индекса

Иногда индекс сам по себе не имеет значения и лучше его просто проигнорировать. Для этого достаточно установить флаг ignore_index=True. Тогда в случае дублирования, кокатенация приведет к созаднию нового целочисленного индекса для итогового объекта Series

In [26]:
x
y
pd.concat([x, y], ignore_index=True)

,A,B
0,A0,B0
1,A1,B1


,A,B
0,A2,B2
1,A3,B3


,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


### Добавление ключей мультииндекса. 

Еще одни вариант - воспользоваться параметром keys для задания меток дял источников данных. Результатом будут иерархически индексированные ряды, содержащие данные:

In [27]:
x
y
pd.concat([x, y], keys=['x', 'y'])

,A,B
0,A0,B0
1,A1,B1


,A,B
0,A2,B2
1,A3,B3


A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

## Конкатенация с использованием соединений

В рассматриваемых примерах в основном производится конкатенация объектов DataFrame с общими названиями столбцов. На практике у данных из разных источников могут быть рзаличные наборы имен столбцов. В этом случае есть два варианта действий

#### Ничего не делать

In [33]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])

df5
df6
pd.concat([df5, df6])

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


,B,C,D
3,B3,C3,D3
4,B4,C4,D4


,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


По умолчанию элементы, данные для которых отсутствуют, заполняются NaN. Чтобы поменять оэто поведение, можно указать одну из нескольких опций параметров join и join_axes функции конкатенации.

#### Указать атрибут join='inner'

По умолчание соединение - объединение входных столбцов (join='outer'), но есть возможность поменять это поведение на пересечение столбцов с помощью опции join='inner'

In [34]:
df5
df6
pd.concat([df5, df6], join='inner')

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


,B,C,D
3,B3,C3,D3
4,B4,C4,D4


,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


# Упращенная конкатенация - .append()

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html - ссылка на документацию

Непосредственная конкатенация массивов настолько распространена, что в объекты Series и DataFrame был включен метод append(), позволяющий выполнить то же самое с меньшими усилиями. Например вместо вызова pd.concat([df1, df2]) можно вызвать df1.append(df2)

Метод .append() можно использовать только для добавления новых строк

In [35]:
df1
df2

df1.append(df2)

,A,B
1,A1,B1
2,A2,B2


,A,B
3,A3,B3
4,A4,B4


,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [36]:
df3
df4

df3.append(df4)

,A,B
0,A0,B0
1,A1,B1


,C,D
0,C0,D0
1,C1,D1


,A,B,C,D
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
0,NaN,NaN,C0,D0
1,NaN,NaN,C1,D1


В отличие от методов .append() и .extend() списков языка Python, метод append() в библиотеке Pandas не изменяет исходный объект. Вместо этого он создает новый объект с объединенными данными, что делает этот метод не слишком эффективным, поскольку означает создание нового индекса и буфера данных. 

Следовательно, если вам необходимо выполнить несколько операций append, лучше создать список объектов DataFrame и передать их все сразу функции concat(). 

# Слияние и соединение  - pd.merge
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html - ссылка на документацию

Одно из важных свойств библиотеки Pandas - ее высокопроизводительные, выполняемые в оперативной памяти операции слияния и соединения. Если вы когда-либо работали с базами данных, вам должен быть знаком такой вид взаимодействия с данными. Основной интерфейс для них - функция pd.merge()

## Соединения "один-к-одному"

In [38]:
df1 = pd.DataFrame({
    'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']
})

df2 = pd.DataFrame({
    'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
    'hire_date': [2004, 2008, 2012, 2014]
})

df1
df2

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [39]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


Функция pd.merge() распознает, что в обоих объектах DataFrame имеется столбец employee, и автоматически выполняет соединение, используя этот столбец в качестве ключа. 

Результатом слияния становится новый объект DataFrame, объединяющий информацию из двух входных объектов.Помимо этого слияние игнорирует индекс, за исключением случаев слияния по индексу, которые будут расмотрены ниже.

## Соединения "многие-к-одному"

Данный вид соединения покрывает случаи, когда один из двух ключевых столбцов содержит дублирующиеся значения. В случае соеднинения "многие-к-одному" в итоговом объекте DataFrame эти дублирующиеся записи будут сохранены.

In [40]:
df4 = pd.DataFrame({
    'group': ['Accounting', 'Engineering', 'HR'],
    'supervisor': ['Carly', 'Guido', 'Steve']
})

In [41]:
df3 # 4 Записи
df4

pd.merge(df3, df4)

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


В итоговом объекте DataFrame имеется дополнительный столбец с информацией о руководителе (supervisor) с повторением информации в одном или нескольких местах в соответствии с вводимыми данными.

## Соединения "многие-ко-многим"

Если столбецключа как в левом, так и в правом массивах содержат повторяющиеся значения, рузультат окажется слиянием типа "многие-ко-многим"

In [42]:
df5 = pd.DataFrame({
    'group': ['Accounting', 'Accounting', 'Engineering', 'Engineering', 'HR', 'HR'],
    'skills': ['math', 'spreadsheets', 'coding', 'linux', 'spreadsheets', 'organization']
})

In [43]:
df1
df5

pd.merge(df1, df5)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


Эти три инструмента можно использвоать и в других инструментах библиотеки Pandas, что дает возможность реализовать широкий диапазон функциональности.

Однако в реальности данные редко бывают такими "чистыми". У функции pd.merge() есть параметры, позволяющие более тонко описывать желаемое поведение операции слияния.

## Задание ключа слияния

По умолчанию pd.merge() выполняет поиск в двух входных объектах соответствующих названий столбцов и использует найденное в качестве ключа. 

Однако имена столбцов могут несовападать побуквенно точно. 

Или у нас может возникнуть потребность объединить данные по конкретному ключу или набору ключей (составному ключу). 

Для подобного случая предусмотренно несколько решений.

### Ключевое слово on

Конкректный ключ или список ключей можно указать с помощью параметра on

In [44]:
df1
df2

pd.merge(df1, df2, on='employee')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


Однако этот параметр работает только в том случае, когда в левом и правом объектах DataFrame имеется указанное название столбца.

### Ключевые слова left_on и right_on

В случаях, когда нужно выполнить слияние двух наборов данны с различными именами столбцов, можно воспользоваться ключевыми словами left_on и right_on, для указания названий двух нужных столбцов

In [49]:
df3 = pd.DataFrame({
    'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 
    'salary': [70_000, 80_000, 120_000, 90_000]
})

df1
df3

pd.merge(df1, df3, left_on="employee", right_on="name")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


Избыточный столбец name, при желании можно удалить с помощью метода .drop()

In [51]:
pd.merge(df1, df3, left_on='employee', right_on='name').drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### Ключевые слова left_index и right_index

Иногда бывает удобнее вместо слияния по столбцу выполнить слияние по индексу. Тогда в pd.merge следует указать флаги left_index и/или right_index:

In [52]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')

df1a
df2a

pd.merge(df1a, df2a, left_index=True, right_index=True)

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


Если требуется комбинация слияния по столбцам и индексам, можно для достижения нужного поведения воспользоваться сочетанием флага left_index с параметром right_on или параметра left_on c флагом right_index:

In [57]:
df1a
df2

pd.merge(df1a, df2, left_index=True, right_on='employee')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


,group,employee,hire_date
1,Accounting,Bob,2008
2,Engineering,Jake,2012
0,Engineering,Lisa,2004
3,HR,Sue,2014


### Метод .join()

Для удобства в объектах DataFrame реализован метод join(), выполняющий по умолчанию слияние по индексам:

In [56]:
df1a
df2a

df1a.join(df2a)

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


## Задание операций над множествами для соединений

Как поступать в случаях, когда какое-либо значение есть в одно ключевом столбце, но отсутствует в другом?

### Внутреннее соеднинение - inner join

In [63]:
df6 = pd.DataFrame({
    'name': ['Peter', 'Paul', 'Mary'],
    'food': ['fish', 'beans', 'bread'],
}, columns=['name', 'food'])

df7 = pd.DataFrame({
    'name': ['Mary', 'Joseph'],
    'drink': ['wine', 'beer']
}, columns=['name', 'drink'])

df6
df7

pd.merge(df6, df7)

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


,name,drink
0,Mary,wine
1,Joseph,beer


,name,food,drink
0,Mary,bread,wine


Здесь мы слили воедино два набора данных, у которых совпадает только одна запись name: Mary. По умолчанию результат будет содержать **пересечение** двух входных множеств - **внутреннее соединение** (inner join). Можно указать это явным образом с помощью ключевого слова how, которое по умолчанию имеет значение 'inner'

In [64]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


### Внешнее соединение - outer join

**Внешнее соединение** означает соединение по **объединению** множеств значений входных столбцов. И заполняет значениями NaN все пропуски значений

In [65]:
df6
df7

pd.merge(df6, df7, how='outer')

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


,name,drink
0,Mary,wine
1,Joseph,beer


,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


### Левое соединение - left join

**Левое соединение** выполняет соединение по значениям левого объекта

In [66]:
df6
df7

pd.merge(df6, df7, how='left')

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


,name,drink
0,Mary,wine
1,Joseph,beer


,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


### Правое соединение - right join

**Правое соединение** выполняет соединение по значениям правого объекта

In [67]:
df6
df7

pd.merge(df6, df7, how='right')

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


,name,drink
0,Mary,wine
1,Joseph,beer


,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


## Пересекающиеся названия столбцов: ключевое слово suffixes

Бывают случаи, когда в двух входных объектах присутсвуют конфликтующие названия столбцов:

In [68]:
df8 = pd.DataFrame({
    'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'rank': [1, 2, 3, 4]
})

df9 = pd.DataFrame({
    'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'rank': [3, 1, 4, 2]
})

df8
df9

pd.merge(df8, df9, on="name")

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Колонка rank присутствует в обоих датафреймах, однако содержит разные значения. Из зва этого возникает конфликт и функция слияния  автоматически добавила в названия суффиксы _x и _y, чтобы обеспечить уникальность названий столбцов результата. 

Если подобное поведение, принятое по умолчанию, неуместно, можно задать пользовательские суффиксы с помощью ключевого слова suffixes:

In [69]:
df8
df9

pd.merge(df8, df9, on='name', suffixes=['_L', '_R'])

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2
